In [ ]:
#ML Final Project Kaggle Competition
#By Jeremiah Pratt and Joseph Bentivegna
#Dedicated to my mother, who always withheld affection and told me I'd never make anything of myself.

In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn import gaussian_process
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import robust_scale
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

In [3]:
trainyBoi = pd.read_csv('train.csv',header=None)
testyBoi = pd.read_csv('test.csv',header=None)

In [ ]:
#Logistic Regression
scaler = MinMaxScaler()
X = trainyBoi.iloc[:,1:]
X = scaler.fit_transform(X)
Y = trainyBoi.iloc[:,0]
kf = KFold(n_splits=5, shuffle=True)
logreg = linear_model.LogisticRegression(penalty='l2',C=10,solver='newton-cg',max_iter=1000,tol=1e-5)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    logreg.fit(X_train,Y_train)
    print(logreg.score(X_test,Y_test))
#print(metrics.confusion_matrix(Y_test,logreg.predict(X_test)))

logreg.fit(X,Y)
testyBoi = scaler.fit_transform(testyBoi)
pred = pd.DataFrame(logreg.predict(testyBoi))
pred.to_csv('results.csv',index_label=['Id','Prediction'],index=list(range(1,200)))

#Playing with feature selection, doesn't really make it better
X = trainyBoi.iloc[:,1:]
sel = MinMaxScaler()
poly = PolynomialFeatures(2)
varthresh = VarianceThreshold(threshold=(.99*(1-.99)))
X = sel.fit_transform(X)
X = poly.fit_transform(X)
X = varthresh.fit_transform(X)
avg = 0
logreg = linear_model.LogisticRegression(penalty='l2',C=50,solver='sag',max_iter=10000,tol=1e-7,multi_class = 'multinomial')
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    logreg.fit(X_train,Y_train)
    print(logreg.score(X_test,Y_test))
    avg = avg + logreg.score(X_test,Y_test)
print(avg/5)

0.59
0.59
0.545
0.555
0.645


C:\Users\jeremiah.v.pratt\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.49


In [ ]:
#Tried Stochastic Gradient Descent
scaler = MinMaxScaler()
poly = PolynomialFeatures(2)
varthres = VarianceThreshold(threshold = (.99*(1-.99)))
X = trainyBoi.iloc[:,1:]
X = scaler.fit_transform(X)
X = poly.fit_transform(X)
Y = trainyBoi.iloc[:,0]
X = varthres.fit_transform(X)
kf = KFold(n_splits=5, shuffle=True)
avg = 0
SGD = linear_model.SGDClassifier(loss="log",penalty="l2",tol=1e-5,max_iter=1000000,fit_intercept=False,alpha=.0001)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    SGD.fit(X_train,Y_train)
    print(SGD.score(X_test,Y_test))
    avg = avg + SGD.score(X_test,Y_test)
#print(metrics.confusion_matrix(Y_test,SGD.predict(X_test)))
print(avg/5)
SGD.fit(X,Y)

In [ ]:
#Playing with feature selection, doesn't really make it better
X = trainyBoi.iloc[:,1:]
Y = trainyBoi.iloc[:,0]
sel = MinMaxScaler()
X=sel.fit_transform(X)
gammas = np.linspace(1/20,1/1000)
cs = np.linspace(.5,50)
tols = [1e-9, 5e-8, 1e-8, 5e-7, 1e-7, 5e-6,1e-6, 5e-5, 1e-5, 5e-4,1e-4,5e-3,1e-3,5e-2,1e-2,5e-1,1e-1,1]
bestavg = 0
bestc = 1000000
besttol = 1000000
bestgam = 10000000
totalRuns = len(gammas)*len(cs)*len(tols)
runs = 0
for i in range(len(gammas)):
    for j in range(len(cs)):
        for k in range(len(tols)):
            clf = svm.SVC(C=cs[j],gamma=gammas[i], decision_function_shape='ovr',tol=tols[k])
            kf = KFold(n_splits=20, shuffle=True)
            avg = 0
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                Y_train, Y_test = Y[train_index], Y[test_index]
                clf.fit(X_train,Y_train)
                #print(logreg.score(X_test,Y_test))
                avg = avg + clf.score(X_test,Y_test)

            avg = avg/20
            if(avg>bestavg):
                bestc = cs[j]
                besttol = tols[k]
                bestgam = gammas[i]
                bestavg = avg

            runs = runs+1
            print("percent done:")
            print(runs/totalRuns)

print(bestavg)
print(besttol)
print(bestc)